## Clustering Zip Codes (Deterministic Multi-Assignment Clustering with Manhattan Distance)

In [1]:
from typing import List
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
import hashlib

### Filters out list of tv stations per zip code based on signal_strength

In [2]:
def filter_signal(data_path: str, signal_strength: List):
    df = pd.read_csv(data_path, usecols=['zipcode', 'callsign', 'signal_strength'], dtype={'zipcode': str})
    cdf = df.groupby(['zipcode', 'callsign'], dropna=False).agg({'signal_strength': min}).unstack(fill_value=0).reset_index()
    cdf.columns = ['zipcode'] + cdf.columns.get_level_values(1).tolist()[1:]
    del cdf[np.nan]
    to_replace = {}
    for ss in [1, 2, 3, 4]:
        to_replace[ss] = 1 if ss in signal_strength else 0
    cdf.replace(to_replace, inplace=True)
    return cdf


### Group zipcodes by manhattan distance

In [3]:
def group_zips(df, diff, n_jobs=8):
    if 'group' in df.columns:
        del df['group']
    if 'hash' in df.columns:
        del df['hash']
    dist = pairwise_distances(df.iloc[:, 1:].values, metric='manhattan', n_jobs=n_jobs)
    x = np.argwhere(dist<=diff)
    group = {}
    for a, b in x:
        d = dist[a, b]
        if a in group: 
            group[a].append({b: d})
        else:
            group[a] = [{b: d}]
    df.loc[:, 'group'] = pd.DataFrame([group]).T
    return df

### Convert each set of zip codes into a hash

In [4]:
def group_hash(c):
    _group_lis = []
    d = eval(c) if type(c) == str else c
    for zipcode_dic in d:
        zipcode = list(zipcode_dic.keys())[0]
        _group_lis.append(zipcode)
    _group_lis.sort()
    _group = ''.join(['{:04d}'.format(x) for x in _group_lis])
    hash_object = hashlib.md5(_group.encode('utf-8'))
    return hash_object.hexdigest()

In [5]:
%%time
data_path = "./stations.csv"
signal_strength = [1, 2]

df = filter_signal(data_path, signal_strength)
df.head()

CPU times: user 1.9 s, sys: 268 ms, total: 2.17 s
Wall time: 2.17 s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZDX,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(df.shape)
# Take out rows with all zeroes
df = df.loc[(df.iloc[:, 1:].sum(axis=1) != 0)]
df.reset_index(drop=True, inplace=True)
print(df.shape)

(36094, 2028)
(33790, 2028)


### Manhattan Distance of 0

In [7]:
%%time
diff = 0
cdf = group_zips(df, diff, n_jobs=12)
cdf.head()

CPU times: user 6h 39min 19s, sys: 18.6 s, total: 6h 39min 38s
Wall time: 42min 40s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]"
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]"
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}]"
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[{3: 0.0}]
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[{4: 0.0}]


In [8]:
%%time
cdf['hash'] = cdf.group.apply(lambda c: group_hash(c))
cdf.head()

CPU times: user 1.04 s, sys: 3.99 ms, total: 1.04 s
Wall time: 1.04 s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group,hash
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]",02e4c3a5a8368e99e66fb97c01fd18da
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]",02e4c3a5a8368e99e66fb97c01fd18da
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}]",fd30b2a33a3297430ff54559211d3a3a
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{3: 0.0}],7cd86ecb09aa48c6e620b340f6a74592
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{4: 0.0}],95b09698fda1f64af16708ffb859eab9


In [10]:
del cdf['group']
cdf.to_pickle("new_diff_0_group_hash.pkl", compression = "xz")
pd.read_pickle("new_diff_0_group_hash.pkl", compression = "xz").head()
#cdf.to_csv("new_diff_0_group_hash.csv", index=False)

,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,hash
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,02e4c3a5a8368e99e66fb97c01fd18da
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,02e4c3a5a8368e99e66fb97c01fd18da
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fd30b2a33a3297430ff54559211d3a3a
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7cd86ecb09aa48c6e620b340f6a74592
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95b09698fda1f64af16708ffb859eab9


In [11]:
df.head()

,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,hash
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,02e4c3a5a8368e99e66fb97c01fd18da
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,02e4c3a5a8368e99e66fb97c01fd18da
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fd30b2a33a3297430ff54559211d3a3a
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7cd86ecb09aa48c6e620b340f6a74592
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95b09698fda1f64af16708ffb859eab9


### Manhattan Distance of 1

In [12]:
%%time
diff = 1
cdf = group_zips(df, diff, n_jobs=12)
cdf.head()

CPU times: user 5h 3min 32s, sys: 5.51 s, total: 5h 3min 38s
Wall time: 34min 8s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]"
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]"
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}, {26: 1.0}]"
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[{3: 0.0}]
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[{4: 0.0}]


In [13]:
%%time
cdf['hash'] = cdf.group.apply(lambda c: group_hash(c))
cdf.head()

CPU times: user 1.89 s, sys: 0 ns, total: 1.89 s
Wall time: 1.89 s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group,hash
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]",02e4c3a5a8368e99e66fb97c01fd18da
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3323: 0.0}]",02e4c3a5a8368e99e66fb97c01fd18da
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}, {26: 1.0}]",80d990a044673ba0cf7dac4e58787e9d
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{3: 0.0}],7cd86ecb09aa48c6e620b340f6a74592
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{4: 0.0}],95b09698fda1f64af16708ffb859eab9


In [14]:
del cdf['group']
cdf.to_pickle("new_diff_1_group_hash.pkl", compression = "xz")
#cdf.to_csv("new_diff_1_group_hash.csv",index=False)

### Manhattan Distance of 2

In [15]:
%%time
diff = 2
cdf = group_zips(df, diff, n_jobs=12)
cdf.head()

CPU times: user 4h 11min 45s, sys: 5.5 s, total: 4h 11min 50s
Wall time: 27min 58s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZMQ,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3322: 2.0}, {3323: 0.0},..."
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3322: 2.0}, {3323: 0.0},..."
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}, {26: 1.0}]"
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[{3: 0.0}]
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[{4: 0.0}, {6: 2.0}, {25: 2.0}, {31: 2.0}, {34..."


In [16]:
%%time
cdf['hash'] = cdf.group.apply(lambda c: group_hash(c))
cdf.head()

CPU times: user 8.59 s, sys: 55.9 ms, total: 8.65 s
Wall time: 8.64 s


,zipcode,K04QP-D,K09YZ-D,K10OG-D,K17ED-D,K17GD-D,K17JI-D,K19GH-D,K20DN-D,K20JX-D,...,WZPX-TV,WZRA-CD,WZRB,WZTV,WZVI,WZVN-TV,WZXZ-CD,WZZM,group,hash
0,00501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3322: 2.0}, {3323: 0.0},...",ea928ad2902b9ca8f8021d0a30ed6682
1,00544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{0: 0.0}, {1: 0.0}, {3322: 2.0}, {3323: 0.0},...",ea928ad2902b9ca8f8021d0a30ed6682
2,00601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{2: 0.0}, {16: 0.0}, {26: 1.0}]",80d990a044673ba0cf7dac4e58787e9d
3,00602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{3: 0.0}],7cd86ecb09aa48c6e620b340f6a74592
4,00603,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[{4: 0.0}, {6: 2.0}, {25: 2.0}, {31: 2.0}, {34...",bb0dbde0d0f900ac4c9566c939c38a40


In [17]:
del cdf['group']
cdf.to_pickle("new_diff_2_group_hash.pkl", compression = "xz")
#cdf.to_csv("new_diff_2_group_hash.csv")